In [8]:
dimension = 'scene'
models = ['cogvideox5b','kling','gen3','lavie','pika','show1','videocrafter2']

In [9]:
# # 把gpt4o_eval的结果加入到annotation中
# import json
# import os


# file_path = "../GPT4o_eval_results/{}/{}_llmeval_new.json".format(dimension,dimension)
# with open(file_path, "r") as file:
#     data = json.load(file)

# video_path ="../Human_anno/{}.json".format(dimension)
# with open(video_path, "r") as file:
#     annotations = json.load(file)

# for index,key in enumerate(data.keys()):
#     for model in models:
#         try:
#             score = int(data[key][model].split(' ')[3][0])
#         except:
#             print(data[key][model],key)
#         annotations[int(key)]['gpt4o_eval'][model] = score

# # for index,key in enumerate(data.keys()):
# #     for line in data[key].split('\n'):
# #         if line.split(' ')[0] != '-':
# #             continue 
# #         else:
# #             try:
# #                 score = int(line.split(' ')[2][0])
# #             except:
# #                 print(line,key)
# #             # score = int(line.split(' ')[2][0])
# #             model =line.split(' ')[1][:-1].lower().replace('**','')
# #             annotations[int(key)]['gpt4o_eval'][model] = score

# with open(video_path, "w") as f:
#     json.dump(annotations, f,indent=4)

In [10]:
# #把baseline的结果加入到annotation中
# import json
# video_path ="../Human_anno/{}.json".format(dimension)
# with open(video_path, "r") as file:
#     annotations = json.load(file)
# for model in models:
#     baseline_path = "../baseline/{}/results_{}_{}_eval_results.json".format(dimension,dimension,model)
#     with open(baseline_path, "r") as file:
#         basescores = json.load(file)
#     basescores = basescores[dimension][1]
#     for i in range(len(annotations)):
#         annotations[i]['baseline_score'][model] = basescores[i]['video_results']

# with open(video_path, "w") as f:
#     json.dump(annotations, f,indent=4)

In [11]:
# # 5->3
# import json
# path ="../Human_anno/humananno_res/scene_2.json"
# with open(path,'r') as f:
#     data = json.load(f)
# for i in range(len(data[dimension].keys())):
#     for model in models:
#         if data[dimension][str(i+1)][model] == 2:
#             data[dimension][str(i+1)][model] = 1
#         elif data[dimension][str(i+1)][model] == 3:
#             data[dimension][str(i+1)][model] = 2
#         elif  data[dimension][str(i+1)][model] == 4 or data[dimension][str(i+1)][model] == 5:
#              data[dimension][str(i+1)][model] = 3
# with open(path,'w') as f:
#     json.dump(data,f,indent=4)

In [12]:
# # 把人类标注数据写入json文件 来源anno_res_anlysis.ipynb
# import json

# import pandas as pd
# anno_path = '../Human_anno/humananno_res/scene_3.json'

# with open(anno_path, 'r') as f:
#     anno_1 = json.load(f)

# jsonpath = '../Human_anno/{}.json'.format(dimension)


# with open(jsonpath, 'r') as f:
#     oc = json.load(f)

# for i in range(len(anno_1[dimension].keys())):
#     for model in models:
#         # oc[i]['human_anno'][model].insert(0, anno_1[dimension][str(i+1)][model])  # 在开头插入数据
#         # oc[i]['human_anno'][model].append(anno_1[dimension][str(i+1)][model])
#         oc[i]['human_anno'][model][3] = anno_1[dimension][str(i+1)][model]

# with open(jsonpath, 'w') as f:
#     json.dump(oc, f, indent=4)

In [13]:
##把错位的gpteval结果重排
# import json
# ph = r'D:\AStudying\AI\Niii_1\hopes\codes\HighlyHumanLikeVEB\GPT4o_eval_results\object_class\object_class_llmeval.json'
# with open(ph, 'r') as f:
#     data = json.load(f)
# # 创建一个字典

# # 按键值大小对字典进行排序
# sorted_dict = dict(sorted(data.items(), key=lambda item: int(item[0])))
# with open(ph, 'w') as f:
#     json.dump(sorted_dict, f, indent=4)

In [14]:
#检查gpt eval与human anno的差异
import json
import numpy as np

scene_json = '../Human_anno/scene.json'
scene_gpt_eval = '../GPT4o_eval_results/scene/scene_llmeval_new.json'
rec_ls = []

with open(scene_json, 'r') as f:
    scene = json.load(f)
with open(scene_gpt_eval, 'r') as f:
    scene_gpt_eval = json.load(f)

models = ['cogvideox5b','gen3', 'kling','videocrafter2', 'pika', 'show1', 'lavie']

idexls = []
for i in range(0,len(scene),3):
    idexls.append(i)
length = len(idexls)

for j in range(length):
    i = idexls[j]
    gpt4o_eval_rs = np.array(list(scene[i]['gpt4o_eval'].values()))
    human_anno = np.array(list(scene[i]['human_anno'].values()))
    baseline_score = np.array(list(scene[i]['baseline_score'].values()))
    anno1 = human_anno[:,0]
    anno2 = human_anno[:,1]

    if gpt4o_eval_rs[-1]>anno1[-1] or gpt4o_eval_rs[-2]>anno2[-2] or gpt4o_eval_rs[-3]>anno1[-3] or gpt4o_eval_rs[-4]>anno2[-4]:
        rec_ls.append(i)
print(len(rec_ls))
for i in rec_ls:
    gpt4o_eval_rs = np.array(list(scene[i]['gpt4o_eval'].values()))
    anno1 = np.array(list(scene[i]['human_anno'].values()))[:,0]
    diff_models = [models[idx] for idx, val in enumerate(gpt4o_eval_rs != anno1) if val]
    print(i+1)
    for model in diff_models:
        print(scene_gpt_eval[str(i)][model])
    print(gpt4o_eval_rs, anno1)


29
4
Final Scores:
- Pika: 3, because the frames clearly depict an amusement park with recognizable elements such as a Ferris wheel, roller coasters, and other rides. The scene is complete and matches the typical features of an amusement park, allowing for easy recognition.
Final Scores:
- Lavie: 3, because the frames clearly depict an amusement park scene with recognizable elements such as a Ferris wheel and other rides. The scene is complete and matches the prompt well.
[3 3 3 3 3 3 3] [3 3 3 3 2 3 2]
7
Final Scores:
- cogvideox5b: 3, because the frames clearly depict an aquarium scene with vibrant coral, various fish, and a turtle, which aligns well with the prompt. The elements are recognizable and consistent with a typical aquarium environment.
Final Scores:
- Gen3: 3, because the frames clearly depict an aquarium scene with various fish and coral, matching the prompt. The underwater environment is recognizable and consistent with what one would expect in an aquarium setting.
Fina